In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
from collections import defaultdict, Counter
from typing import List, Dict, Union
import pickle

import numpy as np
import implicit
from scipy.sparse import lil_matrix
from tqdm import tqdm
import pandas as pd
import polars as pl

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [ ]:
TOP_N = 200
LOCALE = "UK"
VER = "15"
DIR = "/gdrive/MyDrive/amazon_kdd_2023/"

# parameters for IMF
ALPHA = 50000
LAMBDA = 0.01
FACTORS = 200
N_EPOCHS = 15
SEED = 42

In [ ]:
def preprocess(df:pl.DataFrame) -> pl.DataFrame:
    df = df.explode(["prev_items"])
    df = df.with_columns(
        df.select(pl.col("session_id").cumcount().over("session_id").alias("sequence_num"))
    )
    return df

In [ ]:
def train_and_generate_candidates(df:pd.DataFrame):
    # generate sparse matrix
    unique_user_ids = sorted(df["session_id"].unique())
    unique_item_ids = sorted(df["prev_items"].unique())
    user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
    item_id2index = dict(zip(unique_item_ids, range(len(unique_item_ids))))

    matrix = lil_matrix((len(unique_user_ids), len(unique_item_ids)))
    for _, row in tqdm(df.iterrows()):
        user_index = user_id2index[row["session_id"]]
        item_index = item_id2index[row["prev_items"]]
        matrix[user_index, item_index] = 1.0

    # convert LIL to CSR
    matrix = matrix.tocsr()

    # model initialization
    model = implicit.als.AlternatingLeastSquares(
        factors = FACTORS,
        alpha = ALPHA,
        iterations = N_EPOCHS,
        regularization = LAMBDA,
        calculate_training_loss = True,
        random_state = SEED,
    )

    # train model
    model.fit(matrix)

    # generate candidate
    users = []
    candidates = []
    imf_scores = []

    for user_id, user_index in tqdm(user_id2index.items()):
        item_indexes, scores = model.recommend(user_index, matrix[user_index], N=TOP_N, filter_already_liked_items=True)
        for item_index, score in zip(item_indexes, scores):
            users.append(user_id)
            candidates.append(unique_item_ids[item_index])
            imf_scores.append(score)

    candidates = pd.DataFrame({
        "session_id": users,
        "candidate_item": candidates,
        "imf_score": imf_scores,
    })


    candidates = pl.from_pandas(candidates)
    candidates = candidates \
    .sort(["session_id", "imf_score"], descending=[False, True]) \
    .with_columns(pl.col("imf_score").rank(descending=True, method="min").over("session_id").alias("imf_rank"))
    candidates = candidates.to_pandas()


    return model, user_id2index, item_id2index, candidates

## For local train/eval

In [ ]:
train = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task1/train_task1.parquet")
test = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task1/test_task1_phase2.parquet")

train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
session_df = session_df.to_pandas()
session_df = session_df[session_df["locale"] == LOCALE]

In [ ]:
%%time
model, user_id2index, item_id2index, candidates = train_and_generate_candidates(session_df)

5336525it [05:06, 17401.95it/s]


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 1298118/1298118 [5:20:21<00:00, 67.54it/s]


CPU times: user 1d 5h 55min 33s, sys: 12h 3min 3s, total: 1d 17h 58min 36s
Wall time: 5h 42min 36s


In [ ]:
file_name = f"imf_{VER}_{LOCALE}_for_train_or_eval.parquet"
candidates.to_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task1/" + file_name)

In [ ]:
# save models
model_name = f"imf_{VER}_{LOCALE}_model_for_train_or_eval.npz"
user_ids_name = f"imf_{VER}_{LOCALE}_user_id2index_for_train_or_eval.pickle"
item_ids_name = f"imf_{VER}_{LOCALE}_item_id2index_for_train_or_eval.pickle"
model.save(DIR + "models/task1/" + model_name)
with open(DIR + "models/task1/" + user_ids_name, "wb") as f:
    pickle.dump(user_id2index, f)
with open(DIR + "models/task1/" + item_ids_name, "wb") as f:
    pickle.dump(item_id2index, f)

# save candidates
file_name = f"imf_{VER}_{LOCALE}_for_train_or_eval.parquet"
candidates.to_parquet(DIR + "data/interim/candidates/task1/" + file_name)

# For test inference

In [ ]:
train = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task1/train_task1.parquet")
test = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task1/test_task1_phase2.parquet")

In [ ]:
# trainのnext_itemをprev_itemsにappendする
prev_items_list = train["prev_items"].to_list()
next_item_list = train["next_item"].to_list()
prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)

train = train.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

In [ ]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
session_df = session_df.to_pandas()
session_df = session_df[session_df["locale"] == LOCALE]

In [ ]:
%%time
model, user_id2index, item_id2index, candidates = train_and_generate_candidates(session_df)

6518706it [06:12, 17497.73it/s]


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 1298118/1298118 [5:34:45<00:00, 64.63it/s]


CPU times: user 1d 7h 14min 37s, sys: 12h 44min 39s, total: 1d 19h 59min 16s
Wall time: 6h 22s


In [ ]:
# save models
model_name = f"imf_{VER}_{LOCALE}_model_for_inference.npz"
user_ids_name = f"imf_{VER}_{LOCALE}_user_id2index_for_inference.pickle"
item_ids_name = f"imf_{VER}_{LOCALE}_item_id2index_for_inference.pickle"
model.save(DIR + "models/task1/" + model_name)
with open(DIR + "models/task1/" + user_ids_name, "wb") as f:
    pickle.dump(user_id2index, f)
with open(DIR + "models/task1/" + item_ids_name, "wb") as f:
    pickle.dump(item_id2index, f)

# save candidates
file_name = f"imf_{VER}_{LOCALE}_for_inference.parquet"
candidates.to_parquet(DIR + "data/interim/candidates/task1/" + file_name)